In [ ]:
import numpy as np
import csv
import pandas as pd
import pygad
from src import  repeat_elements, tot_TCI_multiple_reactors,  generate_random_list, list_of_potential_number_of_reactors
import matplotlib.pyplot as plt
from scheduleOptimizationForReactorMix import optimize_schedule, capacity_factor_weeks_approach_mix_reactors
import warnings
import time
warnings.filterwarnings('ignore')
import random


In [ ]:
# TCI for a mix of reactors

def tot_TCI_multiple_reactors_mix (power_list, interest_rate, num_reactors_list):
    tot_TCI_for_specific_reactor_power_list = []
    
    for i in range(len(power_list)):
        tot_TCI_for_specific_reactor_power = tot_TCI_multiple_reactors (power_list[i], interest_rate, num_reactors_list[i])
        tot_TCI_for_specific_reactor_power_list.append(tot_TCI_for_specific_reactor_power)
        
    return sum(tot_TCI_for_specific_reactor_power_list)

In [ ]:
def level_cost_of_energy_reactor_mix( interest_rate, power_list, num_reactors_list,\
    list_of_generated_MWh_per_year_from_all_reactors_per_demand, list_of_sold_electricity_MWh_per_year_from_all_reactors, elec_price,\
        list_of_OM_cost_per_year_all_reactors):
    
    sum_cost = 0 # initialization 
    sum_elec = 0 # initialization 
           
    for year in range( len(list_of_generated_MWh_per_year_from_all_reactors_per_demand)):
        if year == 0:
            cap_cost =  tot_TCI_multiple_reactors_mix(power_list, interest_rate, num_reactors_list)
            OM_cost_per_year = 0
            elec_gen = 0
            revenue = 0
        
        elif year > 0:
         
            cap_cost = 0 
            
            OM_cost_per_year =  list_of_OM_cost_per_year_all_reactors[year-1]
            revenue = elec_price * list_of_sold_electricity_MWh_per_year_from_all_reactors[year-1]
            elec_gen =  list_of_generated_MWh_per_year_from_all_reactors_per_demand[year-1]
        
        sum_cost += (cap_cost + OM_cost_per_year - revenue) / ((1 +interest_rate)**(year) ) 
        sum_elec += elec_gen/ ((1 + interest_rate)**year) 
    
    LCOE =  sum_cost/ sum_elec
    # print("LCOE NOW is: ", LCOE)
    return LCOE

# Initialize the solution

In [ ]:
def initial_population_reactors(power_list, demand, sol_per_pop):
    
    rough_multiplier =  int(np.ceil(demand / sum(power_list)))
    initial_list_of_multipliers = [ rough_multiplier-1,  rough_multiplier, rough_multiplier+1]
    
    # Define the length of each list you want to create
    list_length = len(power_list)
    
    
    # Number of lists to create
    num_lists = sol_per_pop
    
    # Store the lists in a container (e.g., a list of lists)
    lists_of_lists = []

    # Create lists
    for _ in range(num_lists):
        # list1 = generate_random_list(min(initial_list_of_multipliers ), max(initial_list_of_multipliers ), list_length)
        list1= [0]*list_length
        lists_of_lists.append(list1)
       
#     # # specify the gene space by limiting the possible number of solitions
#     number_of_reactors_list_of_lists = []
    
#     for i in range(len(power_list )):
#         number_of_reactors_list = list_of_potential_number_of_reactors(power_list[i], demand)
#         number_of_reactors_list_of_lists.append(number_of_reactors_list)
#     # upper_limit_list = []
#     # for i in range(len(power_list )):
#     #     if power_list[i] > 500:
#     #         # I use the multiplier 1.05 because of the ratio between refueling duration and operational lifetime is 0.05
#     #         upper_limit =range(int(np.ceil(1.05*demand/power_list[i])))
#     #     elif power_list[i] <= 500 and power_list[i]>50:  
#     #         upper_limit =range(int(np.ceil(1.03*demand/power_list[i])))
#     #     elif power_list[i] <= 500 and power_list[i]<50:    
#     #          upper_limit =range(int(np.ceil(1.02*demand/power_list[i]))) 
#     #     upper_limit_list.append(upper_limit)    
        
    return lists_of_lists 







def on_gen(ga_instance):
    # pass
    print("Generation : ", ga_instance.generations_completed,  ga_instance.best_solution()[0], ga_instance.best_solution()[1])

def optimize_lcoe(power_list,  levelization_period_weeks, demand , interest_rate, capacity_factor_t_min_criteria):
   
    start_time = time.time()
    
    # GA params
    sol_per_pop = 6*int(np.ceil((len(power_list))) ) 
    
  
    
    
    

    num_generations = 6000
    num_parents_mating = 1 # consider increasing this
    num_genes = len(power_list)
    


    parent_selection_type = "rank"
    keep_parents =  0

    # crossover_type = "single_point"

    mutation_type = "adaptive"
    mutation_percent_genes =  mutation_percent_genes = (20, 1) #10
    
    
    
    
    
    
     
    initial_pop  = initial_population_reactors(power_list, demand, sol_per_pop)
    # potential_solutions = initial_population_reactors(power_list, demand, sol_per_pop)[1]
  
    
    # potential solutions (Gene Space)
    solutions_P_list = []
    for i in range(len(power_list)):
        solutions_P = list_of_potential_number_of_reactors(power_list[i], demand)
        solutions_P_list.append(solutions_P)
    
    allow_dup  = True

    # def fitness_eq(output_discrepancy):
    #     return -100 / (output_discrepancy) 
        
    
    def fitness_func(ga_instance, solution, solution_idx):
       
        power_list_modified = [power_list [i] for i in range(len(solution)) if solution[i] != 0]
      
        Num_of_each_reactor_type_modified = [x for x in solution if x != 0]
      
        long_list_power = repeat_elements(power_list_modified, Num_of_each_reactor_type_modified)
       
        
        if sum(long_list_power) >= demand * capacity_factor_t_min_criteria  and  ((sum(long_list_power))  ) <= 2*demand:
            
            
        
                capacity_factor_results =   capacity_factor_weeks_approach_mix_reactors(long_list_power  ,levelization_period_weeks, demand)
                MWh_generated_per_year_per_demand_list = capacity_factor_results[4]
                MWh_excess_per_year_list =     capacity_factor_results[5]
                Tot_OM__cost_per_year_list =  capacity_factor_results[6]
                
                capacity_factor_t_min = min(capacity_factor_results[1])
               
                output_lcoe = level_cost_of_energy_reactor_mix( interest_rate, power_list_modified, Num_of_each_reactor_type_modified,\
            MWh_generated_per_year_per_demand_list, MWh_excess_per_year_list, 0,\
                Tot_OM__cost_per_year_list)

                
                # if min_tot_P(power_list , solution, levelization_period_weeks) == expected_out :
                # fitness = 1 / ( np.abs(output - expected_out)  +1)
            
                fitness1 =  10000 /(np.abs(output_lcoe - 0))  # Here I assume the target is a very small number (0$/MWh)
                fitness2 =  1.0 / (np.abs(capacity_factor_t_min- 1) + 0.01)
                # if capacity_factor_t_min < capacity_factor_t_min_criteria: # Must satsify the criteria
                #     fitness = -10000

         
         
                
                print("solution is : " , solution)
                print("Total Power is : " , sum(long_list_power))
                print("MW discrepancy : " , np.abs(demand - sum(long_list_power)))
                print("Capacity Factor (min): " , np.round(capacity_factor_t_min, 3))
                print("LCOE : " , np.round(output_lcoe, 3))
            
         
        
        else:
            fitness1 = -100000
            fitness2 = -100000
        if fitness1  > 0 and fitness2 > 0: 
            print("FITNESS NOW: ", [fitness1, fitness2], "\n")
        
        return [fitness1, fitness2]
            
    ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating= num_parents_mating,
                       fitness_func= fitness_func,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                     
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents, # crossover_type=crossover_type,
                       
                       mutation_percent_genes= mutation_percent_genes,
                       gene_space= solutions_P_list,
                       mutation_type=mutation_type, # stop_criteria= ["saturate_20"], # 
                      
                       on_generation= on_gen,
                        fitness_batch_size=1,
                        keep_elitism = int(sol_per_pop/5), # crossover_probability = 0.7,
                        gene_type = int, 
                        initial_population = initial_pop,
                        parallel_processing=8,
                    
                    
                    
                       allow_duplicate_genes=allow_dup )
      
    
 
    ga_instance.run()
  
    end_time = time.time() 
    
    
    sol, sol_fitness, _ = ga_instance.best_solution()
    print("\n The optimization program runtime is " , np.round( (end_time -start_time), 0), " sec", " & The Number of Generations Passed is ",\
        ga_instance.generations_completed, "...... Fitness value of the best solution = {solution_fitness}".format(solution_fitness=sol_fitness)) 


    
    return sol







In [ ]:
# optimize_lcoe( [1000, 300,  50] ,  int(30*365/7), 3000 , 0.06, 0.99)
    

In [4]:
demand_0 = 3000
power_list = [1000, 500, 300, 200, 100, 50, 20, 5, 1]

potential_a1 = list_of_potential_number_of_reactors(power_list[0], demand_0)
potential_a2 = list_of_potential_number_of_reactors(power_list[1], demand_0)
potential_a3 = list_of_potential_number_of_reactors(power_list[2], demand_0)
potential_a4 = list_of_potential_number_of_reactors(power_list[3], demand_0)
potential_a5 = list_of_potential_number_of_reactors(power_list[4], demand_0)
potential_a6 = list_of_potential_number_of_reactors(power_list[5], demand_0)
potential_a7 = list_of_potential_number_of_reactors(power_list[6], demand_0)
potential_a8 = list_of_potential_number_of_reactors(power_list[7], demand_0)
potential_a9 = list_of_potential_number_of_reactors(power_list[8], demand_0)

# potential_a1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# potential_a2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
#                 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
# potential_a3 = list(range(3001))  # Range from 0 to 3000

solutions = []

# Iterate over all combinations of potential solutions
for a1 in potential_a1:
    for a2 in potential_a2:
        for a3 in potential_a3:
             for a4 in potential_a4:
                  for a5 in potential_a5:
                       for a6 in potential_a6:
                            for a7 in potential_a7:
                                for a8 in potential_a8:
                                    for a9 in potential_a9:
                                        a_list = [a9, a8, a7, a6, a5, a4, a3, a2, a1]
                                        a_list1= [i for i in a_list if i != 0]
                                        min_a_list1= min(a_list)
                                        min_a_list_index1 = a_list.index(min_a_list1)
                                        min_P = power_list[min_a_list_index1]
                                        # Check if the combination satisfies the inequality
                                        if demand_0 <= a1 * 1000 + a2 * 500 + a3*300+a4*200 + a5*100 +a6*50+ a7*20+a8*5 + a9*1 <= (demand_0-min_P):
                                            print((a1, a2, a3, a4, a5, a6, a7, a8, a9))
                                            solutions.append((a1, a2, a3, a4, a5, a6, a7, a8, a9))

print("Possible solutions:")
for sol in solutions:
    print(sol)

In [ ]:
["foo", "bar", "bar", "bar"].index("bar")